# Synchrotron Parallel-Beam X-ray CT

This notebook demonstrates how to process and reconstruct parallel-beam tomography data acquired at a synchrotron beamline. Unlike laboratory cone-beam systems, synchrotron setups typically provide a highly collimated parallel beam, enabling more straightforward reconstruction workflows and often higher-quality results.

## 1. Setup and Imports

We begin by importing the required libraries:

- **Core Python tools**:  
  - `h5py`, `os`, and `time` for data handling and file I/O.  
  - `numpy` for numerical operations.  
  - `matplotlib` for plotting.  
  - `tqdm` for progress bars.  

- **nDTomo modules**:  
  - `astra_rec_single`, `astra_rec_vol` for tomographic reconstruction using ASTRA.  
  - `sinocentering`, `scalesinos` for sinogram alignment and scaling.  
  - `cirmask` for applying circular masks to reconstructed slices or volumes.  

- **Algotom**:  
  - `remove_stripe_based_sorting` for ring artefact reduction in sinograms.  

We also enable `%matplotlib qt` to allow interactive plotting in separate windows.  

These tools together provide a complete environment for handling synchrotron CT datasets: from raw sinogram correction through to reconstruction and artefact mitigation.

In [ ]:


import h5py, os, time
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm import tqdm

from nDTomo.tomo.astra_tomo import astra_rec_single, astra_rec_vol
from nDTomo.tomo.sinograms import sinocentering, scalesinos
from nDTomo.methods.misc import cirmask
from algotom.prep.removal import remove_stripe_based_sorting

%matplotlib qt



## 2. Loading Projection Data

We begin by loading the raw projection images from the HDF5 file produced at the synchrotron beamline.

- The dataset path is defined (`p` for the folder and `fn` for the filename).  
- Using **h5py**, we inspect the file structure and confirm the location of the projection data (`entry/data/data`).  
- The projection stack is read into memory as a NumPy array (`proj`).  
- The data are converted to `float32` to reduce memory usage and ensure compatibility with downstream processing.  
- Finally, we print the shape and dtype of the projection array to verify that the data have been loaded correctly.

**Note:**  
The projection data array typically has dimensions corresponding to detector rows, detector columns, and projection angles. Verifying the shape here is essential before constructing sinograms and proceeding with reconstruction.

In [ ]:

p = 'C:\\BIL\\data\\'
fn = 'pco1-140935.hdf'

fn = '%s%s' %(p, fn)

with h5py.File(fn, 'r') as f:
    print(f['entry'].keys())

    print(f['entry/data/data'].shape)

    proj = f['entry/data/data'][:]

proj = proj.astype(dtype='float32')
print(proj.dtype, proj.shape)



## 3. Dark/Flat Correction and Quick Inspection

Before reconstruction, we need to correct the raw projections using dark and flat fields:

1. **Cropping:**  
   Offsets are applied in the x and y directions (`xofs`, `yofs`) to select the central region of the detector. A vertical slice (`ystart:yend`) is chosen to limit the field of view to the relevant part of the sample.  

2. **Dark field:**  
   Computed as the mean of the first 50 frames, where the beam is off. This captures detector background noise.  

3. **Flat field:**  
   Computed as the mean of frames 50–100, where the beam is on but no sample is present. This provides the illumination profile.  

4. **Radiograph:**  
   A single projection (frame 1050) is corrected using the formula:  
   $$
   I_{\text{corr}} = \frac{I_{\text{raw}} - D}{F - D}
   $$

   where \(D\) is the dark field and \(F\) the flat field. Negative values are set to zero.

5. **Visualisation:**  
   - A cropped raw projection for context.  
   - The dark and flat fields separately.  
   - A corrected radiograph to confirm successful normalisation.  
   - A horizontal detector line across multiple projections for quick inspection of intensity variation.  

**Why this matters:**  
Dark/flat correction ensures that detector artefacts and beam inhomogeneities are removed, so that the reconstructed volume reflects only sample absorption and not experimental artefacts.

In [ ]:

xofs = 500
yofs = 660

ystart = 500
yend = 1700

plt.figure(1);plt.clf()
plt.imshow(proj[150,ystart:yend,xofs:-xofs], cmap = 'jet')
plt.show()

dark = np.mean(proj[:50,ystart:yend,xofs:-xofs], axis = 0)
flat = np.mean(proj[50:100,ystart:yend,xofs:-xofs], axis = 0)
radio = proj[1050,ystart:yend,xofs:-xofs]

dark[dark>10000] = 1
flat[flat<10000] = 1

plt.figure(2);plt.clf()
plt.imshow(dark, cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()

plt.figure(3);plt.clf()
plt.imshow(flat, cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()

radio = (radio-dark)/(flat-dark)
radio[radio<0] = 0
print(radio.shape)

plt.figure(4);plt.clf()
plt.imshow(radio, cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()

plt.figure(5);plt.clf()
plt.imshow(proj[100:,-yofs,xofs:-xofs], cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()


## 4. Build Parallel-Beam Sinograms

We now convert the cropped projection stack into **absorption sinograms** suitable for parallel-beam reconstruction:

1. **Select usable projections & crop:**  
   We take frames from index 100 onward and crop to the chosen detector window (`ystart:yend`, `xofs:-xofs`). Any negative raw values are clipped to zero.

2. **Flat/Dark correction + Beer–Lambert transform:**  
   Each projection is corrected as \( (I - D) / (F - D) \).  
   A scaling factor equal to the **detector pixel size** (here $7.9\,\mu\mathrm{m} = 0.0079\,\mathrm{mm}$) is applied, and the Beer–Lambert law is used to convert transmission into line integrals:

   $$
   p = -\ln\!\left(\frac{I - D}{F - D} \times \text{pixel size}\right)
   $$

   Any negative values after the transform are set to zero.

3. **Trim & arrange axes:**  
   The last frame is removed to keep the angular range consistent.  
   The array is then transposed to the order

   $$
   (\text{detector}_x,\ \text{angles},\ \text{detector}_y)
   $$

   which corresponds to `(2, 0, 1)` from the original `(angle, y, x)` layout.  

   This ensures that each detector row produces a standard sinogram with detector \(x\) on one axis and angle on the other.

4. **Quick checks:**  
   - `radios[:, :, 0]` shows a sinogram for the first detector row (detector \(x\) vs. angle).  
   - `radios[:, 0, :]` shows a corrected projection for the first angle (detector \(x\) vs. detector \(y\)).

**Why this matters:**  
Including the pixel size ensures that the sinograms are scaled in **physical units**, which is necessary for accurate parallel-beam reconstruction and for comparing results across different setups.

In [ ]:

radios = np.copy(proj[100:,ystart:yend,xofs:-xofs])
radios[radios<0] = 0
for ii in tqdm(range(radios.shape[0])):    
    radios[ii,:,:] = -np.log(((radios[ii,:,:]-dark)/(flat-dark))*0.079)
radios[radios<0] = 0

radios = radios[:-1,:,:]
radios = np.transpose(radios, (2,0,1))
print(radios.shape)


plt.figure(1);plt.clf()
plt.imshow(radios[:,:,0], cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()

plt.figure(2);plt.clf()
plt.imshow(radios[:,0,:], cmap = 'jet')
plt.colorbar()
plt.axis('tight')
plt.show()


## 5. Preprocessing a Single Sinogram

We now extract and preprocess one representative sinogram for inspection and testing:

1. **Extract sinogram:**  
   We take slice index 120 from the 3D dataset, corresponding to a horizontal cross-section of the sample.

2. **Baseline correction:**  
   The mean value of the first few detector pixels (top rows) is subtracted to remove constant background offset.

3. **Clipping negatives:**  
   Any negative values are set to zero to avoid artefacts in the reconstruction.

4. **Scaling and centering:**  
   - `scalesinos` normalises the sinogram for stability.  
   - `sinocentering` is applied with a search range of ±50 pixels (`crsr=50`) to determine and correct the centre of rotation.  
   - The last column is trimmed to ensure consistency across angles.

5. **Stripe removal:**  
   We apply `remove_stripe_based_sorting` from Algotom to reduce ring artefacts caused by detector inhomogeneities. This operates in the sinogram domain and suppresses vertical stripes that would otherwise appear as rings in the reconstruction.

Finally, we print the processed sinogram shape to confirm dimensions are as expected.

In [ ]:


soi = np.copy(radios[:,:,120])
soi = soi - np.mean(soi[5,:])
soi[soi<0] = 0
soi = scalesinos(soi)
soi = sinocentering(soi, crsr=50)
soi = soi[:,:-1]
soi = np.transpose(remove_stripe_based_sorting(np.transpose(soi), 31))
print(soi.shape)



## 6. Single-Slice Reconstruction

With the preprocessed sinogram ready, we now perform a **filtered back-projection (FBP)** reconstruction of a single slice:

1. **Angles:**  
   The number of projections is taken from the sinogram width. A uniform angular grid from 0° to 180° is generated and converted to radians.

2. **Reconstruction:**  
   - `astra_rec_single` is used to reconstruct the slice.  
   - Negative values in the output (non-physical) are clipped to zero.  
   - A circular mask (`cirmask`) is applied to suppress edge artefacts outside the valid field of view.

3. **Visualisation:**  
   The reconstructed slice is displayed in grayscale for inspection. At this stage we can check for:
   - Correct centering (no double edges or ghosting).  
   - Successful stripe suppression (minimal ring artefacts).  
   - Reasonable contrast across different regions of the sample.

**Why this matters:**  
Running a single-slice reconstruction provides a fast validation step before committing to full-volume reconstruction, ensuring that centering and artefact corrections are working correctly.

In [ ]:


nproj = soi.shape[1]
theta = np.deg2rad(np.linspace(0, 180, nproj, endpoint=False))
fbp = astra_rec_single(soi, theta = theta)
fbp[fbp<0] = 0
fbp = cirmask(fbp, 5)

plt.figure(1);plt.clf()
plt.imshow(fbp, cmap = 'gray', interpolation='none')
plt.show()


## 7. Preprocessing the Full Projection Dataset

After testing on a single slice, we now apply the same corrections to the entire dataset:

1. **Stripe removal:**  
   For each slice in the dataset, `remove_stripe_based_sorting` is applied with a window size of 11 to suppress vertical stripes in the sinograms. This reduces ring artefacts in the final reconstructions.

2. **Baseline correction:**  
   For each slice, the mean of the first few detector rows is subtracted to remove background offset.

3. **Scaling and centering:**  
   - The full dataset is normalised with `scalesinos`.  
   - `sinocentering` is applied with a ±50 pixel search range to refine the centre of rotation across all sinograms.  
   - Linear interpolation is disabled (`interp=False`) for faster correction.  

4. **Clipping negatives:**  
   Any negative values remaining in the dataset are set to zero.

**Why this matters:**  
Performing artefact correction, scaling, and CoR alignment across the whole dataset ensures consistency between slices and prepares the data for reliable 3D reconstruction.

In [ ]:


start = time.time()
for ii in tqdm(range(radios.shape[2])):
    radios[:,:,ii] = np.transpose(remove_stripe_based_sorting(np.transpose(radios[:,:,ii]), 11))
for ii in tqdm(range(radios.shape[2])):
    radios[:,:,ii] = radios[:,:,ii] - np.mean(radios[5,:,ii])
radios = scalesinos(radios)
radios = sinocentering(radios, crsr=50, interp=False)
radios[radios<0] = 0

## 8. Full 3D Reconstruction

With the fully preprocessed sinograms, we now perform a **3D filtered back-projection (FBP)** reconstruction:

1. **Angles:**  
   The number of projections is taken from the dataset shape, and a uniform angular grid spanning 0°–180° is generated in radians.

2. **Reconstruction:**  
   - `astra_rec_vol` reconstructs the entire dataset into a 3D volume.  
   - Negative values are clipped to zero.  
   - A circular mask (`cirmask`) is applied to suppress artefacts outside the valid field of view.

3. **Visualisation:**  
   We display two representative slices of the reconstructed volume:  
   - The **first slice** (`fbp[:, :, 0]`)  
   - The **last slice** (`fbp[:, :, -1]`)  

   Both are shown with a grayscale colormap and a colorbar for intensity reference.

**Why this matters:**  
The full reconstruction provides a volumetric view of the sample’s internal structure. Inspecting slices from opposite ends of the dataset confirms reconstruction stability and highlights any residual artefacts.

In [ ]:

nproj = radios.shape[1]
theta = np.deg2rad(np.linspace(0, 180, nproj, endpoint=False))

fbp = astra_rec_vol(radios, theta=theta)
fbp[fbp<0] = 0
fbp = cirmask(fbp, 5)


plt.figure(1);plt.clf()
plt.imshow(fbp[:,:,0], cmap = 'gray')
plt.colorbar()
plt.show()

plt.figure(2);plt.clf()
plt.imshow(fbp[:,:,-1], cmap = 'gray')
plt.colorbar()
plt.show()

## 9. Saving Processed Data

Finally, we save both the reconstructed volume and the processed sinograms into an HDF5 file for future use and reproducibility:

- **`data`**: the reconstructed 3D volume (`fbp`).  
- **`sinograms`**: the preprocessed sinograms (`radios`).  
- **`pixel_size`**: the detector pixel size in microns (here 7.91 µm).  

Storing the reconstruction, intermediate sinograms, and key metadata in a single file ensures that the dataset can be re-used for further analysis, compared across experiments, or shared as part of the Battery Imaging Library.

In [ ]:

fn = 'C:\\BIL\\data\\processed_data.h5'

with h5py.File(fn, 'w') as f:
    f.create_dataset('data', data = fbp)   
    f.create_dataset('sinograms', data = radios)
    f.create_dataset('pixel_size', data = 7.91)


# Summary

In this notebook we have demonstrated the full workflow for processing and reconstructing **synchrotron parallel-beam X-ray CT data**:

1. Loaded raw projection data from HDF5.  
2. Cropped the field of view and applied dark/flat correction.  
3. Built absorption sinograms using the Beer–Lambert law and pixel size scaling.  
4. Preprocessed the data with baseline subtraction, stripe removal, and centre-of-rotation alignment.  
5. Validated corrections using a single-slice reconstruction.  
6. Reconstructed the entire dataset into a 3D volume using filtered back-projection.  
7. Visualised representative slices to check reconstruction quality.  
8. Saved both the processed sinograms and the reconstructed volume to HDF5 with metadata.  

This pipeline provides a reproducible route from raw synchrotron CT projections to a high-quality 3D reconstruction, suitable for further quantitative analysis or integration into the Battery Imaging Library.